In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import os
import datetime
import math
pd.options.mode.chained_assignment = None
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit

from GPyOpt.methods import BayesianOptimization
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from preprocess.functions.date_inspector import load_files
from eda.functions.eda import show_relative_density_plot, relative_density_plot
from functions.evaluate import EnsembledRegressor, evaluate_idea, nMAE

In [2]:
# regression models
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

reg1 = XGBRegressor(verbose=0)
reg2 = CatBoostRegressor(verbose=0)
reg3 = LGBMRegressor(verbose=0)

In [3]:
df = pd.read_pickle(os.path.abspath(os.path.join(os.getcwd(), '..', 'data','df_forecast_kpx_fe.pkl')))

# Bayesian Optimization
refer : https://github.com/fmfn/BayesianOptimization

In [15]:
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'bayes_opt'

In [ ]:
y = df['Power Generation(kW)+0']#, 'Power Generation(kW)+1','Power Generation(kW)+2']#.values
X = df.drop(['Power Generation(kW)+0','Power Generation(kW)+1','Power Generation(kW)+2','datetime','date','date(forecast)','datetime(forecast)','location'],axis=1)#.values

In [16]:
domain = {'max_bin':(200,300),'learning_rate' : (0.0001,1) , 'n_estimators' : (50,200), 'num_leaves' :(20,100),
          'min_child_weight' : (0.001,1), 'min_data_in_leaf':(30,100)}

In [17]:
def f(max_bin,learning_rate, n_estimators, num_leaves,min_child_weight, min_data_in_leaf) :
    max_bin = int(max_bin)
    
    learning_rate = learning_rate
    n_estimators = int(n_estimators)
    num_leaves = int(num_leaves)
    min_child_weight = min_child_weight
    min_data_in_leaf = int(min_data_in_leaf)

    tscv = TimeSeriesSplit(n_splits=5)
    TimeSeriesSplit(max_train_size=None, n_splits=5)
    result = pd.DataFrame(columns=['train', 'test'])
    result_nmae = pd.DataFrame(columns=['train_nmae','test_nmae'])
    for idx, (train_index, test_index) in enumerate(tscv.split(X)) :
        #print('TRAIN:', train_index.shape[0], 'TEST:', test_index.shape[0])
        x_train, x_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model = LGBMRegressor(max_bin=max_bin, learning_rate=learning_rate, n_estimators=n_estimators,
                              num_leaves=num_leaves, min_child_weight=min_child_weight, min_data_in_leaf=min_data_in_leaf,verbose=0)
        model.fit(x_train, y_train)
        y_train_hat = model.predict(x_train)
        y_test_hat = model.predict(x_test)
        result.loc[idx] = [r2_score(y_train_hat, y_train), r2_score(y_test_hat, y_test)]
        result_nmae.loc[idx] = [nMAE(y_train_hat, y_train), nMAE(y_test_hat, y_test)]

#     display(result)
#     display(result_nmae)
    return -(result_nmae['test_nmae'].iloc[4])

In [ ]:
xgb_bo = BayesianOptimization(
    f = f,
    pbounds = domain,
    verbose=2,
    random_state=1,
)

In [ ]:
xgb_bo.maximize(init_points=3, n_iter=10, acq='ei')